# Rotational dynamics Quickstart

Calculation of rotational dynamics proceeds in two steps. At the first step, we obtain molecular field-free rotational energies, wave functions, and matrix elements of desired electric and magnetic property tensors, such as, for example, dipole moment or polarizability. At the second step, we solve the time-dependent problem with applied external electric and magnetic field using the field-free solutions, obtained in (i), as a basis.

## Molecular field-free rotational states

In [1]:
from richmol.rot.molecule import Molecule, mol_tensor
from richmol.rot.solution import solve

There are three general ways to define the molecule and its properties, using calculated or experimental data or combination of both:

1. Provide Cartesian coordinates of atoms and molecular property tensors, referring to the same coordinate frame, obtained, for example, from quantum chemical calculations.

2. Provide measured spectroscopic constants ($A$, $B$, $C$, $D_{J}$, $D_{J,K}$ ...) and molecular property tensors in the coordinate frame of principal axes of inertia (PAI).

3. Provide measured spectroscopic constants ($A$, $B$, $C$, $D_{J}$, $D_{J,K}$ ...) and calculated molecular property tensors. In this case, the Cartesian coordinates of atoms must also be supplied, which will be used only for rotating the property tensors to the PAI frame.

### Molecular geometry and property tensors

Here is an example for water molecule, using data obtained from a quantum chemical calculation

In [2]:
water = Molecule()

# Cartesian coordinates of atoms
water.XYZ = ("bohr",
             "O",  0.00000000,   0.00000000,   0.12395915,
             "H",  0.00000000,  -1.43102686,  -0.98366080,
             "H",  0.00000000,   1.43102686,  -0.98366080)

# dipole moment (au)
water.dip = [0, 0, -0.7288]

# polarizability tensor (au)
water.pol = [[9.1369, 0, 0], [0, 9.8701, 0], [0, 0, 9.4486]]

for atom in water.XYZ:
    print(atom['label'], atom['mass'], atom['xyz'])

O 15.99491462 [0.         0.         0.06559636]
H 1.0078250322 [ 0.         -0.7572668  -0.52053088]
H 1.0078250322 [ 0.          0.7572668  -0.52053088]


By default, calculations are carried out for the main isotopologue. To specity a non-standard isotope, put the corresponding isotope number next to the atom label, for example, "O18" for oxygen-18 or "H2" for deuterium

In [3]:
# example of D2O^{18}
D2O18 = Molecule()
D2O18.XYZ = ("bohr",
             "O18",  0.00000000,   0.00000000,   0.12395915,
             "H2",   0.00000000,  -1.43102686,  -0.98366080,
             "H2",   0.00000000,   1.43102686,  -0.98366080)

for atom in D2O18.XYZ:
    print(atom['label'], atom['mass'], atom['xyz'])

O18 17.999159613 [0.         0.         0.06559636]
H2 2.0141017781 [ 0.         -0.7572668  -0.52053088]
H2 2.0141017781 [ 0.          0.7572668  -0.52053088]


You can also read/store the geometry from/to XYZ file

In [4]:
# store Cartesian coordinates of atoms into XYZ file
water.store_xyz("water.xyz", "comment line")

# read Cartesian coordinates of atoms from XYZ file
water2 = Molecule()
water2.XYZ = "water.xyz"

for atom in water2.XYZ:
    print(atom['label'], atom['mass'], atom['xyz'])

O 15.99491462 [0.         0.         0.06559636]
H 1.0078250322 [ 0.         -0.7572668  -0.52053088]
H 1.0078250322 [ 0.          0.7572668  -0.52053088]


The molecular frame embedding, i.e., the orientation of $x,y,z$ axes with respect to molecule, can be defined using `frame` property.
The molecule properties `dip` for the dipole moment and `pol` for the polarizability tensor (and few others) will be dynamically rotated to a new coordinate frame whenever the latter changes

In [5]:
# change frame to inertial principal axes system (ipas)
water.frame = "ipas"

# or equivalently
water.frame = "diag(inertia)"

print("coordinatesof atoms\n", water.XYZ)
print("dipole moment\n", water.dip)
print("polarizability\n", water.pol)

print("inertia tensor\n", water.inertia) # must be diagonal in the "ipas" frame

coordinatesof atoms
 [('O', 15.99491462, [ 0.        ,  0.06559636,  0.        ])
 ('H',  1.00782503, [-0.7572668 , -0.52053088,  0.        ])
 ('H',  1.00782503, [ 0.7572668 , -0.52053088,  0.        ])]
dipole moment
 [ 0.     -0.7288  0.    ]
polarizability
 [[9.8701 0.     0.    ]
 [0.     9.4486 0.    ]
 [0.     0.     9.1369]]
inertia tensor
 [[ 0.61496944 -0.         -0.        ]
 [-0.          1.15588059 -0.        ]
 [-0.         -0.          1.77085002]]


Multiple frame operations can be combined together, for example, we can rotate to the inertial principal axes system and then permute the $x$ and $z$ axes

In [6]:
water.frame = "ipas"
water.frame = "zyx"

# or, equivalently in one line
water.frame = "zyx,ipas"

print("coordinates\n", water.XYZ)
print("dipole moment\n", water.dip)
print("polarizability\n", water.pol)

print("inertia tensor\n", water.inertia) # must be diagonal in "ipas" frame

coordinates
 [('O', 15.99491462, [ 0.        ,  0.06559636,  0.        ])
 ('H',  1.00782503, [ 0.        , -0.52053088, -0.7572668 ])
 ('H',  1.00782503, [ 0.        , -0.52053088,  0.7572668 ])]
dipole moment
 [ 0.     -0.7288  0.    ]
polarizability
 [[9.1369 0.     0.    ]
 [0.     9.4486 0.    ]
 [0.     0.     9.8701]]
inertia tensor
 [[ 1.77085002 -0.         -0.        ]
 [-0.          1.15588059 -0.        ]
 [-0.         -0.          0.61496944]]


The principal axes system can be defined with respect to any rank-2 symmetric matrix. In many cases it is convenient to choose a molecular frame in which the polarizability tensor is diagonal

In [7]:
water.frame = "diag(pol)" # can also be combined with axes permutations, e.g., "xzy,diag(pol)"

print("coordinates\n", water.XYZ)
print("dipole moment\n", water.dip)
print("polarizability\n", water.pol)


coordinates
 [('O', 15.99491462, [ 0.        ,  0.06559636,  0.        ])
 ('H',  1.00782503, [ 0.        , -0.52053088, -0.7572668 ])
 ('H',  1.00782503, [ 0.        , -0.52053088,  0.7572668 ])]
dipole moment
 [ 0.     -0.7288  0.    ]
polarizability
 [[9.1369 0.     0.    ]
 [0.     9.4486 0.    ]
 [0.     0.     9.8701]]


We can also define a custom rotation matrix or a principal axes matrix, as demosntrated below

In [8]:
import numpy as np
import scipy

# generate random matrix
mat = np.random.rand(3,3)

# random rotation matrix
water.custom_rot = scipy.linalg.expm((mat - mat.T)/2)

# random symmetric matrix
water.custom_pam = (mat + mat.T)/2

# use 'custom_rot' (orthogonal) matrix to rotate the frame
water.frame = "custom_rot"

# use 'custom_pam' (symmetric) matrix as principal axes matrix
water.frame = "diag(custom_pam)"

Multiple occurences of `frame` assignments lead to accumulation of the corresponding frame rotations. To reset the frame to the initial one, as defined by the input Cartesian coordinates of atoms, we can use `frame=null`

In [9]:
water.frame = "diag(inertia)" # rotate to ipas
water.frame = "zxy" # then permute axes (123)

# now we want to permute axes (23) in the original frame, reset frame rotation
water.frame = "xzy,null"

print(water.XYZ)

[('O', 15.99491462, [ 0.        ,  0.06559636,  0.        ])
 ('H',  1.00782503, [ 0.        , -0.52053088, -0.7572668 ])
 ('H',  1.00782503, [ 0.        , -0.52053088,  0.7572668 ])]


As already mentioned, only certain molecule properties (such as `dip`, `pol`, `inertia`, `XYZ`, and few others) are automatically rotated when the frame is altered. This is demonstrated in the following example

In [10]:
water.vec = [1,2,3]
water.mat = [[1,2,3],[4,5,6],[7,8,9]]

water.frame = "zyx,null"

# axes permutation (13) does not affect vec and mat, but dip
print("vec\n", water.vec)
print("mat\n", water.mat)
print("dip\n", water.dip)

vec
 [1, 2, 3]
mat
 [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
dip
 [-0.7288  0.      0.    ]


If you want to declare custom Cartesian tensor which is dynamically rotated to a new frame, use `mol_tensor` function

In [11]:
water.vec = mol_tensor([1,2,3])
water.mat = mol_tensor([[1,2,3],[4,5,6],[7,8,9]])

water.frame = "zyx,null"

# axes permutation (13) now affects also vec and mat
print("vec\n", water.vec)
print("mat\n", water.mat)
print("dip\n", water.dip)

# we can then change vec and mat values, they will be still dynamically rotated
water.vec = [7,8,9]
water.mat = mol_tensor([[11,12,13],[14,15,16],[17,18,19]])

print("new vec\n", water.vec)
print("new mat\n", water.mat)

vec
 [3. 2. 1.]
mat
 [[9. 8. 7.]
 [6. 5. 4.]
 [3. 2. 1.]]
dip
 [-0.7288  0.      0.    ]
new vec
 [9. 8. 7.]
new mat
 [[19. 18. 17.]
 [16. 15. 14.]
 [13. 12. 11.]]


### Rotational constants

The rotational constants (in units of cm$^{-1}$) can be calculated from the input molecular geometry using `ABC` property. This will work only if the molecular frame is set to the inertia principal axes system or if the inertia tensor is diagonal

In [12]:
water.frame = "diag(pol)"
print(water.ABC) # works because inertia tensor is diagonal in the polarizability principal axes system

water.frame = "diag(inertia)"
print(water.ABC) # the constants in inertia frame will be sorted

[9.51951209577557, 14.584229929162115, 27.41213978457678]
[27.41213978457678, 14.584229929162115, 9.51951209577557]


The values of rotational constants can also be user specified, for example, using the experimental values. In these case, `ABC` will return the user-defined values. If we want to access the rotational constants calculated from molecular geometry, we should use `ABC_calc`

In [13]:
print("calculated ABC", water.ABC)
water.ABC = (9.285, 14.512, 27.877) # user-defined rotational constants in units of cm^-1
print("user input ABC ", water.ABC)
print("calculated ABC", water.ABC_calc)

calculated ABC [27.41213978457678, 14.584229929162115, 9.51951209577557]
user input ABC  [27.877, 14.512, 9.285]
calculated ABC [27.41213978457678, 14.584229929162115, 9.51951209577557]


***************************

**Important**: When user-defined rotational constants specified, the rotational Hamiltonian is built using these values

***************************

For linear and spherical-top molecules, use `B` to print or assign the rotational constant

In [14]:
ocs = Molecule()
ocs.B = 0.20286 # experimental value
print(ocs.B)

ocs.XYZ = ("angstrom", "C", 0, 0, 0, "S", 0, 0, -1.56, "O", 0, 0, 1.16)
print(ocs.B_calc) # calculated value

print(ocs.linear()) # True if molecule is linear

0.20286
0.20317858059669255
True
/home/andrey/.local/lib/python3.6/site-packages/richmol-0.1a1-py3.6-linux-x86_64.egg/richmol/rot/molecule.py:216: RuntimeWarning: divide by zero encountered in double_scalars
  abc = [convert_to_cm/val for val in np.diag(itens)]


The input rotational constants are always checked against the calculated once, if molecular geometry is specified. Here is an example of an error produced when molecular frame is not in the inertia principal axes system

In [15]:
water2 = Molecule()
water2.ABC = (9.285, 14.512, 27.877) # user-defined rotational constants in units of cm^-1

water2.XYZ = ("bohr",
              "O",  0.00000000,   0.00000000,   0.12395915,
              "H",  0.00000000,  -1.43102686,  -0.98366080,
              "H",  0.00000000,   1.43102686,  -0.98366080)

# polarizability tensor (au)
water2.pol = [[9.1369, 0, 0], [0, 9.8701, 0], [0, 0, 9.4486]]

water2.frame = 'ipas' # inertia principal axes system
print("inertia frame")
print("calculated ABC", water2.ABC_calc)
print("input ABC     ", water2.ABC)

water2.frame = 'diag(pol)' # polarizability principal axes system
print("\npolarizability frame")
print("calculated ABC", water2.ABC_calc)
print("input ABC     ", water2.ABC)

inertia frame
calculated ABC [27.41213978457678, 14.584229929162115, 9.51951209577557]
input ABC      [27.877, 14.512, 9.285]

polarizability frame
calculated ABC [9.51951209577557, 14.584229929162115, 27.41213978457678]


ValueError: input experimental rotational constants differ much from the calculated once
        exp          calc       exp-calc
A    27.877000     9.519512    18.357488
B    14.512000    14.584230    -0.072230
C     9.285000    27.412140   -18.127140

### Rotational solutions 

The rotational energies and wave functions can be calculated for a specified range of $J$ quantum number using the function `solve`

In [16]:
sol = solve(water, Jmin=0, Jmax=10)

The state energies and assignments can be printed out as following

In [17]:
for J,sol_J in sol.items():
    for symmetry,sol_sym in sol_J.items():
        for istate in range(sol_sym.nstates):
           print(J, symmetry, istate, sol_sym.enr[istate], sol_sym.assign[istate])

0 A 0 0.0 ['0' '0' '0' ' 1.000000']
1 A 0 23.797000000000004 ['1' '0' '1' ' 1.000000']
1 A 1 37.16199999999999 ['1' '1' '0' ' 1.000000']
1 A 2 42.38899999999999 ['1' '1' '1' ' 1.000000']
2 A 0 70.13332800101738 ['2' '0' '0' ' 0.981068']
2 A 1 79.52900000000001 ['2' '1' '1' ' 1.000000']
2 A 2 95.20999999999998 ['2' '1' '0' ' 1.000000']
2 A 3 135.30499999999995 ['2' '2' '1' ' 1.000000']
2 A 4 136.56267199898258 ['2' '2' '0' ' 0.981068']
3 A 0 136.90947734903332 ['3' '0' '1' ' 0.922382']
3 A 1 142.36908432351558 ['3' '1' '0' ' 0.995098']
3 A 2 173.5352392423415 ['3' '1' '1' ' 0.992048']
3 A 3 206.69599999999994 ['3' '2' '0' ' 1.000000']
3 A 4 212.56852265096663 ['3' '2' '1' ' 0.922382']
3 A 5 287.29891567648446 ['3' '3' '0' ' 0.995098']
3 A 6 287.4947607576584 ['3' '3' '1' ' 0.992048']
4 A 0 222.36947823747218 ['4' '0' '0' ' 0.834399']
4 A 1 225.06755539025477 ['4' '1' '1' ' 0.982779']
4 A 2 276.01478677087727 ['4' '1' '0' ' 0.962955']
4 A 3 300.89169764043777 ['4' '2' '1' ' 0.994878']
4 

The energies are in units of cm$^{-1}$ and the assignment is a list of numbers ($J$, $k$, $\tau$, $c$) denoting the leading symmetric-top function in the expansion of wave function. $\tau$ is the pairty of rotational state, defined as $(-1)^\tau$, and $c$ is the absolute value of the leading coefficient.

We can print more than one leading contribution in the assignment

In [18]:
for J,sol_J in sol.items():
    for symmetry,sol_sym in sol_J.items():
        sol_sym.assign_nprim = 3 # print three leading contributions
        for istate in range(sol_sym.nstates):
           print(J, symmetry, istate, sol_sym.enr[istate], sol_sym.assign[istate])

0 A 0 0.0 ['0' '0' '0' ' 1.000000']
1 A 0 23.797000000000004 ['1' '0' '1' ' 1.000000' '1' '1' '0' ' 0.000000' '1' '1' '1' ' 0.000000']
1 A 1 37.16199999999999 ['1' '1' '0' ' 1.000000' '1' '0' '1' ' 0.000000' '1' '1' '1' ' 0.000000']
1 A 2 42.38899999999999 ['1' '1' '1' ' 1.000000' '1' '0' '1' ' 0.000000' '1' '1' '0' ' 0.000000']
2 A 0 70.13332800101738 ['2' '0' '0' ' 0.981068' '2' '2' '0' ' 0.018932' '2' '1' '0' ' 0.000000']
2 A 1 79.52900000000001 ['2' '1' '1' ' 1.000000' '2' '1' '0' ' 0.000000' '2' '0' '0' ' 0.000000']
2 A 2 95.20999999999998 ['2' '1' '0' ' 1.000000' '2' '1' '1' ' 0.000000' '2' '2' '0' ' 0.000000']
2 A 3 135.30499999999995 ['2' '2' '1' ' 1.000000' '2' '0' '0' ' 0.000000' '2' '1' '0' ' 0.000000']
2 A 4 136.56267199898258 ['2' '2' '0' ' 0.981068' '2' '0' '0' ' 0.018932' '2' '1' '0' ' 0.000000']
3 A 0 136.90947734903332 ['3' '0' '1' ' 0.922382' '3' '2' '1' ' 0.077618' '3' '1' '0' ' 0.000000']
3 A 1 142.36908432351558 ['3' '1' '0' ' 0.995098' '3' '3' '0' ' 0.004902' '3' 

### Symmetry

The molecular symmetry can be specified using `sym` molecule property, for example, the above calculation can be done using the $D_2$ or $C_{2v}$ rotational symmetry groups

In [28]:
water.sym = 'D2'
sol_d2 = solve(water, Jmin=0, Jmax=3)

water.sym = 'C2v'
sol_c2v = solve(water, Jmin=0, Jmax=3)

print("D2 solutions")
for J,sol_J in sol_d2.items():
    for symmetry,sol_sym in sol_J.items():
        for istate in range(sol_sym.nstates):
           print(J, symmetry, istate, sol_sym.enr[istate], sol_sym.assign[istate])

print("C2v solutions")
for J,sol_J in sol_c2v.items():
    for symmetry,sol_sym in sol_J.items():
        for istate in range(sol_sym.nstates):
           print(J, symmetry, istate, sol_sym.enr[istate], sol_sym.assign[istate])

D2 solutions
0 A 0 0.0 ['0' '0' '0' ' 1.000000']
1 B1 0 23.797000000000004 ['1' '0' '1' ' 1.000000']
1 B2 0 30.161999999999995 ['1' '1' '0' ' 1.000000']
1 B3 0 35.388999999999996 ['1' '1' '1' ' 1.000000']
2 A 0 36.92675042240616 ['2' '2' '0' ' 0.935448']
2 A 1 73.7692495775938 ['2' '0' '0' ' 0.935448']
2 B1 0 39.304999999999964 ['2' '2' '1' ' 1.000000']
2 B2 0 80.20999999999997 ['2' '1' '0' ' 1.000000']
2 B3 0 64.52899999999998 ['2' '1' '1' ' 1.000000']
3 A 0 62.69599999999996 ['3' '2' '0' ' 1.000000']
3 B1 0 57.86958074813088 ['3' '2' '1' ' 0.946217']
3 B1 1 147.60841925186907 ['3' '0' '1' ' 0.946217']
3 B2 0 -172.76620748256204 ['3' '3' '0' ' 0.998773']
3 B2 1 116.43420748256207 ['3' '1' '0' ' 0.998773']
3 B3 0 -172.73150116617208 ['3' '3' '1' ' 0.999002']
3 B3 1 147.76150116617208 ['3' '1' '1' ' 0.999002']
C2v solutions
0 A1 0 0.0 ['0' '0' '0' ' 1.000000']
1 A2 0 23.797000000000004 ['1' '0' '1' ' 1.000000']
1 B1 0 30.161999999999995 ['1' '1' '0' ' 1.000000']
1 B2 0 35.38899999999999

We can restrict solutions to have certain symmetries, by specifying `only` argument to `solve`

In [30]:
Jmin = 0
Jmax = 5

water.sym = 'D2'
sym = ['B2', 'B1']
sym_filter = {J : sym for J in range(Jmin, Jmax+1)}
sol_d2 = solve(water, Jmin=0, Jmax=3, only={'sym':sym_filter})

water.sym = 'C2v'
sym = ['A2', 'B2']
sym_filter = {J : sym for J in range(Jmin, Jmax+1)}
sol_c2v = solve(water, Jmin=0, Jmax=3, only={'sym':sym_filter})

print("D2 solutions")
for J,sol_J in sol_d2.items():
    for symmetry,sol_sym in sol_J.items():
        for istate in range(sol_sym.nstates):
           print(J, symmetry, istate, sol_sym.enr[istate], sol_sym.assign[istate])

print("C2v solutions")
for J,sol_J in sol_c2v.items():
    for symmetry,sol_sym in sol_J.items():
        for istate in range(sol_sym.nstates):
           print(J, symmetry, istate, sol_sym.enr[istate], sol_sym.assign[istate])

D2 solutions
1 B1 0 23.797000000000004 ['1' '0' '1' ' 1.000000']
1 B2 0 30.161999999999995 ['1' '1' '0' ' 1.000000']
2 B1 0 39.304999999999964 ['2' '2' '1' ' 1.000000']
2 B2 0 80.20999999999997 ['2' '1' '0' ' 1.000000']
3 B1 0 57.86958074813088 ['3' '2' '1' ' 0.946217']
3 B1 1 147.60841925186907 ['3' '0' '1' ' 0.946217']
3 B2 0 -172.76620748256204 ['3' '3' '0' ' 0.998773']
3 B2 1 116.43420748256207 ['3' '1' '0' ' 0.998773']
C2v solutions
1 A2 0 23.797000000000004 ['1' '0' '1' ' 1.000000']
1 B2 0 35.388999999999996 ['1' '1' '1' ' 1.000000']
2 A2 0 39.304999999999964 ['2' '2' '1' ' 1.000000']
2 B2 0 64.52899999999998 ['2' '1' '1' ' 1.000000']
3 A2 0 57.86958074813088 ['3' '2' '1' ' 0.946217']
3 A2 1 147.60841925186907 ['3' '0' '1' ' 0.946217']
3 B2 0 -172.73150116617208 ['3' '3' '1' ' 0.999002']
3 B2 1 147.76150116617208 ['3' '1' '1' ' 0.999002']


### Centrifugal distortion rotational constants

The centrifugal distortion constants can used to build Watson-type asymmetric top Hamiltonian in the A or S standard reduced form (J. K. G. Watson in "Vibrational Spectra and Structure" (Ed: J. Durig) Vol 6 p 1, Elsevier, Amsterdam, 1977)

* A-form: $H_A = H_\text{rigrot} - D_{J} J^{4} - D_{JK} J^{2} J_{z}^{2} - D_{K} J_{z}^{4} - \frac{1}{2}  [ d_{1} J^{2} + d_{2} J_{z}^{2}, J_{+}^{2} + J_{-}^{2} ]_{+} + H_{J} J^{6} + H_{JK} J^{4} J_{z}^{2} + H_{KJ} J^{2} J_{z}^{4} + H_{K} J_{z}^{6} + \frac{1}{2} [ h_{1} J^{4} + h_{2} J^{2} J_{z}^{2} + h_{3} J_{z}^{4}, J_{+}^{2} + J_{-}^{2} ]_{+}$

* S-form: $H_S = H_\text{rigrot} - D_{J} J^{4} - D_{JK} J^{2} J_{z}^{2} - D_{K} J_{z}^{4} + d_{1} J^{2} (J_{+}^{2} + J_{-}^{2}) + d_{2} (J_{+}^{4} + J_{-}^{4}) + H_{J} J^{6} + H_{JK} J^{4} J_{z}^{2} + H_{KJ} J^{2} J_{z}^{4} + H_{K} J_{z}^{6} + h_{1} J^{4} (J_{+}^{2} + J_{-}^{2}) + h_{2} J^{2} (J_{+}^{4} + J_{-}^{4}) + h_{3} (J_{+}^{6} + J_{-}^{6})$

The type of Watson Hamiltonian can be specified using `watson` molecule property

In [20]:
water.watson = 'watson_a'
water.dj = 0
water.djk = 2
water.dk = 3

sol = solve(water, Jmin=0, Jmax=5, verbose=True) # set verbose=True to check which attributes of water are used to build Watson Hamiltonian

solve for J = 0 and symmetry A1
build rigid-rotor Hamiltonian from rotational constants, A, B, C = (27.877, 14.512, 9.285) cm-1
add 'watson_s' term 'dj'
add 'watson_s' term 'djk'
add 'watson_s' term 'dk'
solve for J = 1 and symmetry A2
build rigid-rotor Hamiltonian from rotational constants, A, B, C = (27.877, 14.512, 9.285) cm-1
add 'watson_s' term 'dj'
add 'watson_s' term 'djk'
add 'watson_s' term 'dk'
solve for J = 1 and symmetry B1
build rigid-rotor Hamiltonian from rotational constants, A, B, C = (27.877, 14.512, 9.285) cm-1
add 'watson_s' term 'dj'
add 'watson_s' term 'djk'
add 'watson_s' term 'dk'
solve for J = 1 and symmetry B2
build rigid-rotor Hamiltonian from rotational constants, A, B, C = (27.877, 14.512, 9.285) cm-1
add 'watson_s' term 'dj'
add 'watson_s' term 'djk'
add 'watson_s' term 'dk'
solve for J = 2 and symmetry A1
build rigid-rotor Hamiltonian from rotational constants, A, B, C = (27.877, 14.512, 9.285) cm-1
add 'watson_s' term 'dj'
add 'watson_s' term 'djk'
add '

### Matrix elements of laboratory-frame Cartesian tensor operators

Once we have solutions of the field-free rotational problem (using `solve`), we can compute the matrix elements of various molecule-field interaction tensors, which we generally call as laboratory-frame Cartesian tensor operators

In [23]:
from richmol.rot.labtens import LabTensor

# matrix elements of lab-frame dipole moment operator
dipole_moment = LabTensor(water.dip, sol)

# matrix elements of lab-frame polarizability tensor
polarizability = LabTensor(water.pol, sol)

In [25]:
print(dipole_moment)
print(polarizability)